- GPT - 2
  - Transformer 디코더 블럭을 여러개 쌓아서 만들었음
    - 1.셀프어텐션 레이어
      - 입력한 문장에서 단어들 간의 관계를 파악해 중요한 단어에 집중하도록 고려
    - 2. Feed Forward Neural Network
      - 각 단어 벡터를 독립적으로 변환해서 테스트의 특징을 추출
    - 3. Residual Connection & Layer Normalization
      - 각 레이어의 입력과 출력을 합산해서 안정성은 높이고 학습이 빠름
  - Self-Attention
    - Query @ key --> 어텐션 스코어 * 벨류 --> 가중합 - >softmax 확률분포
  - Positional Encoding
    - Recurrent 구조가 아님.. 단어의 순서를 구하기위서 위치정보사용(사인 코사인 정보를 번갈아가면서 사용해서)
  - Masked Self-Attention
    - 미래정보를 사용하지 않기위해서 이전단어의 정보만 사용
  - Auto-regressive 구조
    - 이전 단어를 기반으로 해서 다음 단어를 예측하는 문장을 생성
  - 레이어 : Residual, normalization

- 전체적인 동작
  - 입력 문장을 토큰화해서 임베딩 벡터로 변환
  - Postional Encoding 을 더해서 위치정보를 포함한상태 각 디코더 블럭에 전달
  - 각 디코더 블럭의 self-attention 레이어에서 단어 간의 관계를 계산
  - attention output -> Feed-Forward Network 변환됨
  - 이 과정을 디코더 블럭에서 반복 최종출력은 각 디코더가 출력한 확률에서 가장 높은 단어를 출력



# 책에있는 코드가 아닌 일반적인 방법

In [1]:
# 데이터 로드
import gdown
file_id = '1EJdu56msbDuR1Awd4w9nnutIWCsaeAey'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'fineTune_data.txt', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1EJdu56msbDuR1Awd4w9nnutIWCsaeAey
To: c:\Github\python_ML\LLM\fineTune_data.txt
100%|██████████| 24.6k/24.6k [00:00<00:00, 833kB/s]


'fineTune_data.txt'

In [4]:
import torch
import torch_directml
# dml = torch_directml.device()
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AdamW
from transformers import get_linear_schedule_with_warmup

# 모델과 토크나이저 로드
MODEL_NAME = 'skt/kogpt2-base-v2'
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)  #Hugging Face 불러옴

# 패딩 토큰 설정
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # 시퀀스를 동일한 길이로 맞추기 위해서 사용

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME) # 사전학습된 모델로드(전이학습)
model.resize_token_embeddings(len(tokenizer)) # 패딩토큰 추가했으니. 임배딩층 재 조정

# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.input_ids = []
        for text in texts:
            encodings_dict = tokenizer(text, truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.input_ids[idx]

# 텍스트 데이터를 로드하고 전처리
with open('./fineTune_data.txt', 'r', encoding='utf-8') as f:
    texts = [line.strip() for line in f if line.strip()]

# 데이터셋 및 데이터로더 설정
dataset = TextDataset(texts, tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 학습 설정
device = torch_directml.device() # torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(dataloader) * 3  # 3 epochs
# 초기에는 낮은학습률로 시작해서 점점 학습률을 높이는 방식
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

c:\Users\Playdata\AppData\Local\anaconda3\envs\pytdml\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm
# 파인튜닝 루프
model.train()
for epoch in range(10):
  iterator = tqdm(dataloader)
  for batch in iterator:
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)

      model.zero_grad()

      outputs = model(inputs, labels=labels)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      scheduler.step()
      iterator.set_description(f"epoch:{epoch+1} Loss: {loss.item():.3f}")

# 파인튜닝 된 모델 저장
model.save_pretrained('fine_tuned_kogpt2')
tokenizer.save_pretrained('fine_tuned_kogpt2')

print("모델 파인튜닝이 완료되었습니다.")

epoch:7 Loss: 4.704:  78%|███████▊  | 28/36 [01:09<00:19,  2.45s/it]

In [ ]:
texts[0]

'그때에 김첨지는 대수롭지 않은듯이,'

In [ ]:
# 파인튜닝된 모델과 토크나이져 로드
model = GPT2LMHeadModel.from_pretrained('fine_tuned_kogpt2')
tokenizer = GPT2TokenizerFast.from_pretrained('fine_tuned_kogpt2')

In [ ]:
# 입력할 문장
input_text = '폭풍우'
# 토큰화
input_ids = tokenizer.encode(input_text,return_tensors='pt')
# gpu or cpu 모델을 전송
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = input_ids.to(device)
# 텍스트 생성 - 역전파가 필요없으므로 no_grad
with torch.no_grad():
  generated_ids = model.generate(input_ids, max_length=128
                              ,num_return_sequences=1  # 생성할 텍스트의 수
                              ,repetition_penalty=2.0
                              ,pad_token_id=tokenizer.pad_token_id
                              ,eos_token_id=tokenizer.eos_token_id
                              # ,bos_token_id=tokenizer.bos_token_id  # GPT2 필요없지만 시작문장을 강조
                              ,use_cache=True
                              )
  # 토큰을 텍스트로 디코딩
  generated_text = tokenizer.decode(generated_ids[0],skip_special_tokens=True)
  print(generated_text)

[9324, 8364, 9272]